In [1]:
import os
import re
import numpy as np
import pandas as pd
from pathlib import Path
from copy import deepcopy
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import ward, fcluster, dendrogram, linkage

In [2]:
from openpyxl import load_workbook
import xlsxwriter

In [3]:
%%time
INP_DNAME = os.getcwd()
data_fname = '2020 Trade Show Account Matching Worksheet.xlsx'

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 12.4 µs


In [4]:
# "All Clients" table dataframe
df_all = pd.read_excel(data_fname, sheet_name='All Clients & Prospect Accts')

In [5]:
# Specific table dataframe
df_sheet_1 = pd.read_excel(data_fname, sheet_name='Feb 20 Coterie')

In [6]:
wb = load_workbook('2020 Trade Show Account Matching Worksheet.xlsx')

In [7]:
list_hupetlink_account = []
for i in range(2, (len(df_all)+2)):
    list_hupetlink_account.append(wb['All Clients & Prospect Accts']['C'+str(i)].value)

In [8]:
df_all['Link'] = list_hupetlink_account

In [9]:
df_all.head(2)

,Parent Account,Account Name,Account ID,Parent Account ID,Account Owner,Account Record Type,Account Currency,Website,Link
0,NaN,Fast Retailing,001i000000UL9l4,NaN,Kate O'Brien,NuORDER Client,USD,NaN,"=HYPERLINK(""https://nuorder.my.salesforce.com/..."
1,Trimera Group Inc,Gottex Swimwear Brands,0013100001lE5vO,0013100001ZJuUR,Jessica Ramsden,NuORDER Client,USD,http://www.gottexmodels.com,"=HYPERLINK(""https://nuorder.my.salesforce.com/..."


In [10]:
df_sheet_1.head(2)

,Exhibitor,Fuzzy Match Account Name,Account ID,Parent Account ID,Account Owner,Account Record Type,Trade Show ID,Booth,New,Show
0,10 Eleven,NaN,001i000000AjeKp,NaN,NaN,NaN,a1I3r000006YHyMEAW,NaN,NaN,COTERIE
1,143 Tees,NaN,0010Z00002Ce1k3,NaN,NaN,NaN,a1I3r000006YHyMEAW,7427 - Level 3,NaN,COTERIE


# Clearing and changing data

In [35]:
# Creating customer name lists
list_all_name = list(df_all['Account Name'])
list_sheet_name = list(df_sheet_1['Exhibitor'])

In [36]:
list_all_name[402]

'Bond-Eye Australia Pty Ltd'

In [37]:
# Function to convert client names to strings
def func_list_name(list_name):
    for i in range(len(list_name)):
        list_name[i] = str(list_name[i])
    return list_name

In [38]:
# List building
list_all_name = func_list_name(list_all_name)
list_sheet_name = func_list_name(list_sheet_name)

In [39]:
list_all_name[402]

'Bond-Eye Australia Pty Ltd'

In [40]:
def name_lower(list_name):
    for i in range(len(list_name)):
        list_name[i] = list_name[i].lower()
    return list_name

In [41]:
list_all_name_l = name_lower(list_all_name)
list_sheet_name_l = name_lower(list_sheet_name)

In [42]:
list_all_name_l[402]

'bond-eye australia pty ltd'

In [43]:
# Function for removing punctuation and abbreviations
def clear_list_name(list_name):
    for i in range(len(list_name)):
        list_name[i] = re.sub(r'\W+', ' ', list_name[i])
        list_name[i] = re.sub(r'\Wco\W|\Winc\W|\Wltd\W|\Wcorp\W|\Wllc\W|\Wplc\W|\Wldc\W|\Wsa\W|\Wbv\W|\Wag\W|\Wslg\W|\Wrsn\W|\Wmfg\W|\Wsrl\W|\Wpty\W|\Wspa\W|\Wsagl\W|\Wdtp\W', ' ', list_name[i])
        list_name[i] = re.sub(r'\Wco\b|\Winc\b|\Wltd\b|\Wcorp\b|\Wllc\b|\Wplc\b|\Wldc\b|\Wsa\b|\Wbv\b|\Wag\b|\Wslg\b|\Wrsn\b|\Wmfg\b|\Wsrl\b|\Wpty\b|\Wspa\b|\Wsagl\b|\Wdtp\b', ' ', list_name[i])    
        list_name[i] = re.sub(r'\W+', '', list_name[i])
    return list_name

In [44]:
# List building
list_all_name = clear_list_name(list_all_name)
list_sheet_name = clear_list_name(list_sheet_name)

In [45]:
list_all_name[402]

'bondeyeaustralia'

In [46]:
# Match function
def find_match(list_sheet_name, list_all_name, sheet_name_match):
    for i in range(len(list_sheet_name)):
        if list_sheet_name[i] in list_all_name:
            sheet_name_match.append(list_all_name.index(list_sheet_name[i]))
        else:
            sheet_name_match.append(0)
    return sheet_name_match

In [57]:
hash('Keds') + hash('Kids') == hash('Kids') + hash('Keds')

True

In [ ]:
hash('Kids Keds')

In [47]:
# Listing indices of matching rows
sheet_name_match = []
sheet_name_match = find_match(list_sheet_name, list_all_name, sheet_name_match)

In [49]:
# Data Markup Block
list_account = []
list_fuzzy = []
list_link = []
for i in range(len(sheet_name_match)):
    if sheet_name_match[i] == 0:
        list_account.append('')
        list_fuzzy.append('')
        list_link.append('')
    else:
        list_account.append(df_all['Account ID'].iloc[sheet_name_match[i]])
        list_fuzzy.append('TRUE MATCH')
        list_link.append(df_all['Link'].iloc[sheet_name_match[i]])

# Preparation for clustering. Complete match list

In [50]:
df_result = deepcopy(df_sheet_1)

In [51]:
len(df_result)

1844

In [52]:
len(sheet_name_match)

1844

In [53]:
df_result['Account ID'] = list_account
df_result['Fuzzy Match Account Name'] = list_fuzzy

In [54]:
df_result.head()

,Exhibitor,Fuzzy Match Account Name,Account ID,Parent Account ID,Account Owner,Account Record Type,Trade Show ID,Booth,New,Show
0,10 Eleven,TRUE MATCH,001i000000AjeKp,NaN,NaN,NaN,a1I3r000006YHyMEAW,NaN,NaN,COTERIE
1,143 Tees,TRUE MATCH,0010Z00002Ce1k3,NaN,NaN,NaN,a1I3r000006YHyMEAW,7427 - Level 3,NaN,COTERIE
2,209 Wst 38,,,NaN,NaN,NaN,a1I3r000006YHyMEAW,NaN,NaN,COTERIE
3,27 Miles Malibu,,,NaN,NaN,NaN,a1I3r000006YHyMEAW,7447 - Level 3,NaN,COTERIE
4,2wins,,,NaN,NaN,NaN,a1I3r000006YHyMEAW,NaN,New Exhibitor,COTERIE


In [27]:
df_result['Parent Account ID'] = df_result['Parent Account ID'].fillna('')
df_result['Account Owner'] = df_result['Account Owner'].fillna('')
df_result['Account Record Type'] = df_result['Account Record Type'].fillna('')

In [28]:
df_result.head()

,Exhibitor,Fuzzy Match Account Name,Account ID,Parent Account ID,Account Owner,Account Record Type,Trade Show ID,Booth,New,Show
0,10 Eleven,TRUE MATCH,001i000000AjeKp,,,,a1I3r000006YHyMEAW,NaN,NaN,COTERIE
1,143 Tees,TRUE MATCH,0010Z00002Ce1k3,,,,a1I3r000006YHyMEAW,7427 - Level 3,NaN,COTERIE
2,209 Wst 38,,,,,,a1I3r000006YHyMEAW,NaN,NaN,COTERIE
3,27 Miles Malibu,,,,,,a1I3r000006YHyMEAW,7447 - Level 3,NaN,COTERIE
4,2wins,,,,,,a1I3r000006YHyMEAW,NaN,New Exhibitor,COTERIE


# Output

In [29]:
workbook = xlsxwriter.Workbook('result_new.xlsx', {'nan_inf_to_errors': True})

In [30]:
worksheet = workbook.add_worksheet('Feb 20 Coterie')

In [31]:
# tst_list = []
# for i in range(len(list_link)):
#     tst_list.append(list_account[i] + ':' + list_link[i])

In [32]:
# tst_list = clear_list_name(tst_list)

In [33]:
# for i in range(len(tst_list)):
#     tst_list[i] = tst_list[i].replace("=HYPERLINK(",'')

In [34]:
# tst_list[0].split(',')

In [35]:
# for i in range(len(df_result)):
#     for j in range(7):
#         if j == 2:
#             worksheet.write_formula('C'+str(i), list_link[i])
#         else:
#             worksheet.write(i, j, df_result.iloc[i][j])

In [36]:
for i in range(len(df_result)):
    for j in range(len(df_sheet_1.columns)):
        if i == 0:
            if j == 0:
                worksheet.write(i, j, 'Exhibitor Name')
            if j == 1:
                worksheet.write(i, j, 'Fuzzy Match Account Name')
            if j == 2:
                worksheet.write(i, j, 'Account ID')
            if j == 3:
                worksheet.write(i, j, 'Parent Account ID')
            if j == 4:
                worksheet.write(i, j, 'Account Owner')
            if j == 5:
                worksheet.write(i, j, 'Account Record Type')
            if j == 6:
                worksheet.write(i, j, 'Trade Show ID')
        else:
            if j == 2 and i > 0:
                worksheet.write_formula('C'+str(i), list_link[i])
            else:
                worksheet.write(i, j, df_result.iloc[i][j])

In [37]:
# # for j in range(len(df_sheet_1.columns)):
# #     worksheet.write(0, j, str(df_sheet_1.columns[j]))
    
# for i in range(len(df_result)):
#     for j in range(len(df_sheet_1.columns)):
#         if j == 2:
#             if list_link[i] == '':
#                 worksheet.write(i, j, ' ')
#             else:
#                 worksheet.write_formula('C'+str(i + 1), list_link[i])
#         else:
#             worksheet.write(i + 1, j, df_result.iloc[i][j])

In [38]:
workbook.close()

In [208]:
list_link[2] 

''

In [102]:
df_result.iloc[0][0]

'& Sons Garment Co.'

In [103]:
list_link[2]

''